In [ ]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests
import json

rimfire = "https://www.luckygunner.com/rimfire"
handgun = "https://www.luckygunner.com/handgun"
rifle = "https://www.luckygunner.com/rifle"
shotgun = "https://www.luckygunner.com/shotgun"

In [ ]:
# returns 2 list of http links separated by availability
def luckyGunner_availability(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    products = soup.find_all('div', class_='main')

    
    # list of all handgun ammo seperated by availability
    products = soup.find_all('div', class_='block-content block-no-padding')
    bulletsList = str(products).split('<div class="main-category-cols">')
    inStock = bulletsList[1]
    outofStock = bulletsList[2]

    inStockScrapedLinks = []
    outofStockScrapedLinks = []
    
    inStock = inStock.split("</li>")
    for inStockItems in inStock:
        inStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(inStockItems)))


    outofStock = outofStock.split("</li>")
    for outofStockItems in outofStock:
        outofStockScrapedLinks.append(re.findall("(?P<url>https?://[^\s]+)", str(outofStockItems)))

    inStockCleanLinks = []
    outofStockCleanLinks = []
    

    for i in inStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                    
                else:
                    inStockCleanLinks.append(cleanLink)
                    break
                
    for i in outofStockScrapedLinks:
        cleanLink = ""
        for j in i:
            for k in j:
                if k != '"':
                    cleanLink += k
                else:
                    outofStockCleanLinks.append(cleanLink)
                    break
    return inStockCleanLinks, outofStockCleanLinks

handGunAmmoInStock, handGunAmmoOutofStock = luckyGunner_availability(handgun)
rifleAmmoInStock, rifleAmmoOutofStock = luckyGunner_availability(rifle)

In [ ]:
def scrapeHandGunAmmo(link):
    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body
    products = soup.find_all('div', class_='main')

    names = []
    prices = []

    # list of html elements
    names_list = soup.find_all("h3")
    prices_list = soup.find_all("div", class_="price-box")

    
    for i in prices_list:

        # if 
        if "Special Price" in i:
            sp = i.find("p", class_="special-price")
            sp = sp.find("span", class_="price")
            prices.append(sp.text)
        else:
            prices.append(i.text)


    for i in  names_list:
        names.append(i.text)

    try:
        ammoData = pd.DataFrame(
            {"name": names, "price": prices}
        )
        return ammoData
        
    except:
        print("Check link")

In [ ]:
def scrapeRifleAmmo(link):

    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body    

    prices = []
    names = []

    # list of html elements
    names_list = soup.find_all("h3")
    prices_list = soup.find_all("div", class_="price-box")

    
    for i in prices_list:

        # if 
        if "Special Price" in i:
            sp = i.find("p", class_="special-price")
            sp = sp.find("span", class_="price")
            prices.append(sp.text)
        else:
            prices.append(i.text)


    for i in  names_list:
        names.append(i.text)

    try:
        ammoData = pd.DataFrame(
            {"name": names, "price": prices}
        )
        return ammoData
        
    except:
        print("Check link")
        

    print("\n\n------------------------------------\n\n")

In [ ]:
def scrapeShotgunAmmo(link):

    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    page_body = soup.body    

    prices = []
    names = []
    dirty_links = []
    clean_links = []

    # list of html elements
    names_list = soup.find_all("h3")
    gauges = soup.find_all("ul", class_="dropdown-menu")

    try:
        for i in gauges[3]:
            aLink = i.find("a")
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(aLink))
            if len(aLink) != 0:
                dirty_links.append(aLink)
            
    except:
        print(-1)

    for i in dirty_links:
        clean_links.append(i[0].strip('"'))


    return clean_links

In [ ]:
def scrapeShotgunAmmo_2(link):

    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    names = []
    prices = []

    name = soup.find_all("h3")

    
    for i in name:
        names.append(i.text)

    prices_list = soup.find_all("div", class_="price-box")
    for i in prices_list:
        if "Special Price" in i:
            sp = i.find("p", class_="special-price")
            sp = sp.find("span", class_="price")
            prices.append(sp.text)
        else:
            prices.append(i.text)

    try:
        ammoData = pd.DataFrame(
            {"name": names, "price": prices}
        )
        return ammoData
    except:
        print("check link: " + link)

In [ ]:
def scrapeRimfire(link):

    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")
    names = []
    prices = []
    dirty_links = []
    clean_links = []

    caliber = soup.find_all("ul", class_="dropdown-menu")
    for i in caliber[2]:
        aLink = i.find("a")
        aLink = re.findall("(?P<url>https?://[^\s]+)", str(aLink))
        if len(aLink) != 0:
            dirty_links.append(aLink)
        
    for i in dirty_links:
        aLink = i[0].strip('"')
        clean_links.append(aLink)
        
    return clean_links

In [ ]:
def scrapeRimefireAmmo_2(link):
    link+="?limit=all"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    html_response = requests.get(url=link, headers = headers)
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "lxml")

    names = []
    prices = []
    name = soup.find_all("h3")
    price = soup.find_all("div", class_="price-box")
    for i in name:
        names.append(i.text)
    

    for i in price:
        if "Special Price" in i:
            sp = i.find("p", class_="special-price")
            sp = sp.find("span", class_="price")
            prices.append(sp.text)
        else:
            prices.append(i.text)

    try:
        ammoData = pd.DataFrame(
            {"name": names, "price": prices}
        )
        return ammoData
    except:
        print("check link: " + link)

In [ ]:
rimfireAmmo = []
rimfire_links = scrapeRimfire(rimfire)
for i in range(len(rimfire_links)-1):
    df = scrapeRimefireAmmo_2(rimfire_links[i])
    rimfireAmmo.append(df)

In [ ]:
shotgun_ammo = []
shotgun_links = scrapeShotgunAmmo(shotgun)
for i in range(len(shotgun_links)-1):
    shotgun_ammo.append(scrapeShotgunAmmo_2(shotgun_links[i]))

In [ ]:
handgunAmmo = []
for availableAmmoLink in handGunAmmoInStock:
    df = scrapeHandGunAmmo(availableAmmoLink)
    handgunAmmo.append(df)

In [ ]:
rifleAmmo = []
for availableAmmoLink in rifleAmmoInStock:
    df = scrapeRifleAmmo(availableAmmoLink)
    rifleAmmo.append(df)

In [ ]:
import os, shutil, glob
jsonStr = ""
jsonStr += "["
aCount = 0
for i in rifleAmmo:
    res = i.to_json(orient="split")
    parsed = json.loads(res)
    file = json.dumps(parsed)  
    jsonStr += str(file)
    
    if aCount == len(rifleAmmo)-1:
        continue
    else:
        jsonStr += ","
        aCount += 1

jsonStr += "]"
d = json.loads(jsonStr)

savePath = "/Users/chakaneshegog/Desktop/Ammo_web/webScrape/jsonData/LuckyGunner"
with open(os.path.join(savePath, "LGRifleAmmo.json"), "w") as fp:
    json.dump(d, fp)

In [ ]:
jsonStr = ""
jsonStr += "["
aCount = 0
for i in handgunAmmo:
    res = i.to_json(orient="split")
    parsed = json.loads(res)
    file = json.dumps(parsed)  
    jsonStr += str(file)
    
    if aCount == len(handgunAmmo)-1:
        continue
    else:
        jsonStr += ","
        aCount += 1

jsonStr += "]"


savePath = "/Users/chakaneshegog/Desktop/Ammo_web/webScrape/jsonData/LuckyGunner"
with open(os.path.join(savePath, "LGHandgunAmmo.json"), "w") as fp:
    json.dump(d, fp)

In [ ]:
jsonStr = ""
jsonStr += "["
aCount = 0
for i in shotgun_ammo:
    res = i.to_json(orient="split")
    parsed = json.loads(res)
    file = json.dumps(parsed)  
    jsonStr += str(file)
    
    if aCount == len(shotgun_ammo)-1:
        continue
    else:
        jsonStr += ","
        aCount += 1

jsonStr += "]"


savePath = "/Users/chakaneshegog/Desktop/Ammo_web/webScrape/jsonData/LuckyGunner"
with open(os.path.join(savePath, "LGShotgunAmmo.json"), "w") as fp:
    json.dump(d, fp)

In [ ]:
jsonStr = ""
jsonStr += "["
aCount = 0
for i in rimfireAmmo:
    res = i.to_json(orient="split")
    parsed = json.loads(res)
    file = json.dumps(parsed)  
    jsonStr += str(file)
    
    if aCount == len(rimfireAmmo)-1:
        continue
    else:
        jsonStr += ","
        aCount += 1

jsonStr += "]"


savePath = "/Users/chakaneshegog/Desktop/Ammo_web/webScrape/jsonData/LuckyGunner"
with open(os.path.join(savePath, "LGRimfireAmmo.json"), "w") as fp:
    json.dump(d, fp)